In [9]:
#!pip install requests
#!pip install pandas
!pip install bs4


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import sys
print(sys.executable)

/Users/baoyunhang/Desktop/Hackthon/.venv/bin/python


In [11]:
#! /usr/local/bin/python3 -m pip install requests pandas beautifulsoup4
! pip install typing


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import random
import logging
from urllib.parse import urlparse
import socket
import http.client
from playwright.async_api import async_playwright, Browser, Page
delay = random.uniform(2, 5)  # 随机延时 2-5 秒
from typing import Dict, Optional
time.sleep(delay)

In [13]:
def scrape_seo_signals(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status() 
        soup = BeautifulSoup(response.text, 'html.parser')
        
        #Title tags
        title = soup.title.string if soup.title else "No Title Found"

        # Meta Description
        meta_description = ""
        meta = soup.find("meta", attrs={"name": "description"})
        if meta:
            meta_description = meta.get("content", "")

        # H1 tag
        h1_tags = [h1.get_text(strip=True) for h1 in soup.find_all("h1")]

        #sitemap
        sitemap_exists = soup.find("a", href=True, text=lambda x: "sitemap" in x.lower()) is not None

        #
        images = soup.find_all("img")
        image_alts = [img.get("alt", "No Alt") for img in images]

        # HTTPS
        https_used = url.startswith("https")

        # Schema Markup
        structured_data = bool(soup.find("script", type="application/ld+json"))

        # mobile friendly
        viewport = soup.find("meta", attrs={"name": "viewport"})
        mobile_friendly = viewport is not None

        # loading time
        load_time = response.elapsed.total_seconds()

        # Canonical tag
        canonical_tag = soup.find("link", rel="canonical")
        canonical_url = canonical_tag['href'] if canonical_tag else "No Canonical Tag"

        # robots meta tag
        robots_meta = ""
        robots = soup.find("meta", attrs={"name": "robots"})
        if robots:
            robots_meta = robots.get("content", "")

        
        internal_links = []
        external_links = []
        for a_tag in soup.find_all("a", href=True):
            href = a_tag["href"]
            if url in href or href.startswith("/"):
                internal_links.append(href)
            else:
                external_links.append(href)

       
        body_content = soup.body.get_text(separator=" ", strip=True) if soup.body else ""
        word_count = len(body_content.split())

        
        keyword = "law"
        keyword_density = body_content.lower().count(keyword) / len(body_content.split()) if body_content else 0

        
        favicon = soup.find("link", rel="icon") or soup.find("link", rel="shortcut icon")
        favicon_exists = favicon["href"] if favicon else "No Favicon"

        
        og_title = soup.find("meta", property="og:title")
        og_description = soup.find("meta", property="og:description")
        og_image = soup.find("meta", property="og:image")
        social_tags = {
            "OG Title": og_title["content"] if og_title else "No OG Title",
            "OG Description": og_description["content"] if og_description else "No OG Description",
            "OG Image": og_image["content"] if og_image else "No OG Image",
        }

        
        broken_links = 0
        for link in external_links + internal_links:
            try:
                link_response = requests.head(link, headers=headers, timeout=5)
                if link_response.status_code >= 400:
                    broken_links += 1
            except:
                broken_links += 1

        
        core_web_vitals = {"LCP": "N/A", "FID": "N/A", "CLS": "N/A"}

        
        return {
            "URL": url,
            "Title": title,
            "Meta Description": meta_description,
            "H1 Tags": h1_tags,
            "Sitemap Exists": sitemap_exists,
            "HTTPS Used": https_used,
            "Image Alts": image_alts,
            "Structured Data": structured_data,
            "Mobile Friendly": mobile_friendly,
            "Load Time (s)": load_time,
            "Canonical Tag": canonical_url,
            "Robots Meta Tags": robots_meta,
            "Internal Links": len(internal_links),
            "External Links": len(external_links),
            "Word Count": word_count,
            "Keyword Density": keyword_density,
            "Favicon Exists": favicon_exists,
            "OG Tags": social_tags,
            "Broken Links": broken_links,
            "Core Web Vitals": core_web_vitals,
        }

    except Exception as e:
        return {"URL": url, "Error": str(e)}

    
def read_urls_from_file(file_path):
    df = pd.read_csv(file_path)
    urls_to_scrape = df[df['in_progress'] == 0].head(10)
    return urls_to_scrape, df


def update_progress(df, index, output_file):
    df.at[index, 'in_progress'] = 1
    df.to_csv(output_file, index=False)


def main():
    input_file = r"/Users/baoyunhang/Desktop/parcaticum/export_url_file.csv"
    output_file = r"/Users/baoyunhang/Desktop/parcaticum/export_url_file_updated.csv"

    try:
        urls_to_scrape, df = read_urls_from_file(input_file)
    except Exception as e:
        print(f"Error reading input file: {e}")
        return

    if urls_to_scrape.empty:
        print("No URLs to process. All URLs are marked as completed.")
        return

    results = []
    delay = 2

    for index, row in urls_to_scrape.iterrows():
        url = row['link']
        print(f"Scraping: {url}")
        seo_data = scrape_seo_signals(url)
        results.append(seo_data)

        update_progress(df, index, output_file)

        time.sleep(delay)

    results_df = pd.DataFrame(results)
    results_df.to_csv(r"/Users/baoyunhang/Desktop/parcaticum/export_url_file.csv", index=False)
    print("All URLs processed. Results saved to seo_signals_results.csv")

In [14]:
class AsyncSEOScraper:
    def __init__(self):
        self.playwright = None
        self.browser: Optional[Browser] = None
        self.context = None

    async def init_browser(self):
        """初始化浏览器"""
        try:
            if self.browser:
                await self.browser.close()
            
            self.playwright = await async_playwright().start()
            self.browser = await self.playwright.chromium.launch(
                headless=True,
                args=[
                    '--disable-blink-features=AutomationControlled',
                    '--disable-dev-shm-usage',
                    '--no-sandbox',
                ]
            )
            
            self.context = await self.browser.new_context(
                user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
                viewport={'width': 1920, 'height': 1080}
            )
        except Exception as e:
            logging.error(f"Error initializing browser: {str(e)}")
            raise

    async def get_performance_metrics(self, page: Page):
        """"Obtain performance indicators"""
        try:
            metrics = await page.evaluate("""() => {
                const timing = window.performance.timing;
                const navigationStart = timing.navigationStart;
                return {
                    'Load Time': (timing.loadEventEnd - navigationStart) / 1000,
                    'DOM Content Load': (timing.domContentLoadedEventEnd - navigationStart) / 1000,
                    'First Paint': performance.getEntriesByType('paint')[0]?.startTime / 1000 || 'N/A'
                }
            }""")
            return metrics
        except:
            return {'Load Time': 'N/A', 'DOM Content Load': 'N/A', 'First Paint': 'N/A'}

    async def check_link_status(self, page: Page, link: str, base_url: str) -> bool:
        """Check link status"""
        try:
            full_url = urljoin(base_url, link)
            response = await page.context.request.head(full_url)
            return response.ok
        except:
            return False

    async def scrape_url(self, url: str, retry_count: int = 3) -> Dict:
        """crawl single url signal"""
        for attempt in range(retry_count):
            page = None
            try:
                if not self.browser:
                    await self.init_browser()
                
                page = await self.context.new_page()
                await page.set_extra_http_headers({
                    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                    'Accept-Language': 'en-US,en;q=0.5',
                    'Accept-Encoding': 'gzip, deflate, br',
                    'DNT': '1'
                })
                
                start_time = time.time()
                response = await page.goto(url, wait_until='networkidle')
                load_time = time.time() - start_time
                
                if response and response.status == 403:
                    raise Exception("403 Forbidden")
                
                # wait page load
                await page.wait_for_load_state('networkidle')
                content = await page.content()
                soup = BeautifulSoup(content, 'html.parser')

                # basic seo information
                title = soup.title.string.strip() if soup.title else "No Title Found"
                meta = soup.find("meta", attrs={"name": "description"})
                meta_description = meta.get("content", "").strip() if meta else "No Description Found"
                
                # H1tag
                h1_tags = [h1.get_text(strip=True) for h1 in soup.find_all("h1")]
                
                # picture alt text
                images = soup.find_all("img")
                image_alts = [img.get("alt", "No Alt") for img in images]
                
                # Schema sign
                structured_data = bool(soup.find("script", type="application/ld+json"))
                
                # mobile_friendly
                viewport = soup.find("meta", attrs={"name": "viewport"})
                mobile_friendly = viewport is not None
                
                # tag
                canonical_tag = soup.find("link", rel="canonical")
                canonical_url = canonical_tag['href'] if canonical_tag else "No Canonical Tag"
                
                # Robots meta标签
                robots = soup.find("meta", attrs={"name": "robots"})
                robots_meta = robots.get("content", "") if robots else "No Robots Meta"
                
                # link analysis
                internal_links = []
                external_links = []
                domain = urlparse(url).netloc
                for a_tag in soup.find_all("a", href=True):
                    href = a_tag["href"]
                    if domain in href or href.startswith("/"):
                        internal_links.append(href)
                    else:
                        external_links.append(href)
                
                # content analysis
                body_content = soup.body.get_text(separator=" ", strip=True) if soup.body else ""
                word_count = len(body_content.split())
                
                # keywords density
                keyword = "lawyer"  # 针对律师网站
                keyword_density = body_content.lower().count(keyword) / word_count if word_count > 0 else 0
                
                # social tag
                og_title = soup.find("meta", property="og:title")
                og_description = soup.find("meta", property="og:description")
                og_image = soup.find("meta", property="og:image")
                
                # performance metric
                performance_metrics = await self.get_performance_metrics(page)
                
                return {
                    "URL": url,
                    "Title": title,
                    "Meta Description": meta_description,
                    "H1 Tags": h1_tags,
                    "HTTPS Used": url.startswith("https"),
                    "Image Alts Count": len([alt for alt in image_alts if alt != "No Alt"]),
                    "Total Images": len(images),
                    "Structured Data Present": structured_data,
                    "Mobile Friendly": mobile_friendly,
                    "Load Time (s)": performance_metrics['Load Time'],
                    "DOM Content Load (s)": performance_metrics['DOM Content Load'],
                    "First Paint (s)": performance_metrics['First Paint'],
                    "Canonical Tag": canonical_url,
                    "Robots Meta": robots_meta,
                    "Internal Links Count": len(internal_links),
                    "External Links Count": len(external_links),
                    "Word Count": word_count,
                    "Keyword Density": f"{keyword_density:.2%}",
                    "Has OG Title": og_title is not None,
                    "Has OG Description": og_description is not None,
                    "Has OG Image": og_image is not None,
                    "Status": "Success"
                }
                
            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed for {url}: {str(e)}")
                if attempt == retry_count - 1:
                    return {
                        "URL": url,
                        "Status": f"Failed: {str(e)}"
                    }
                await asyncio.sleep(random.uniform(20, 30))
                await self.init_browser()
            finally:
                if page:
                    await page.close()

    async def close(self):
        """关闭浏览器和playwright"""
        if self.browser:
            await self.browser.close()
        if self.playwright:
            await self.playwright.stop()

async def run_scraper():
    """using async run_scraper"""
    input_file = r"/Users/baoyunhang/Downloads/top_10_results_filtered.csv"
    output_file = r"/Users/baoyunhang/Downloads/top_10_results_filtered_updated.csv"
    results_file = r"/Users/baoyunhang/Desktop/parcaticum/sf_top10_seo_signals_results.csv"
    
    try:
        df = pd.read_csv(input_file)
        if 'in_progress' not in df.columns:
            df['in_progress'] = 0
        
        urls_to_scrape = df[df['in_progress'] == 0].head(1)
        
        if urls_to_scrape.empty:
            logging.info("No URLs to process. All URLs are marked as completed.")
            return
        
        scraper = AsyncSEOScraper()
        results = []
        
        try:
            for index, row in urls_to_scrape.iterrows():
                url = row['link']
                logging.info(f"Processing: {url}")
                
                result = await scraper.scrape_url(url)
                results.append(result)
                
                df.at[index, 'in_progress'] = 1
                df.to_csv(output_file, index=False)
                
                await asyncio.sleep(random.uniform(15, 25))
            
            results_df = pd.DataFrame(results)
            
            try:
                existing_results = pd.read_csv(results_file)
                combined_results = pd.concat([existing_results, results_df], ignore_index=True)
                combined_results.to_csv(results_file, index=False)
            except FileNotFoundError:
                results_df.to_csv(results_file, index=False)
            
        finally:
            await scraper.close()
        
        logging.info("Processing completed successfully")
        
    except Exception as e:
        logging.error(f"An error occurred in main: {str(e)}")

def main():
    asyncio.get_event_loop().run_until_complete(run_scraper())

# if __name__ == "__main__":
#     main()

In [ ]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright, Browser, Page
import pandas as pd
import random
import logging
from bs4 import BeautifulSoup
from typing import Optional, Dict
import time
from urllib.parse import urljoin, urlparse
import os
import sys
from datetime import datetime

# setting more consice log
log_file = f'scraper_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler(sys.stdout)
    ]
)

# activate nest_asyncio
nest_asyncio.apply()

#[keep former AsyncSEOScraper class code unchanged...]

async def run_scraper():
    
    input_file = r"/Users/baoyunhang/Downloads/top_10_results_filtered.csv"
    output_file = r"/Users/baoyunhang/Downloads/top_10_results_filtered_updated.csv"
    results_file = r"/Users/baoyunhang/Desktop/parcaticum/sf_top10_seo_signals_results.csv"
    
    try:
        # read input_file
        logging.info(f"Reading input file: {input_file}")
        df = pd.read_csv(input_file)
        total_urls = len(df)
        logging.info(f"Successfully read {total_urls} rows from input file")
        
        if 'in_progress' not in df.columns:
            df['in_progress'] = 0
            logging.info("Added 'in_progress' column to DataFrame")
        
        # obtain all undisposed data
        urls_to_scrape = df[df['in_progress'] == 0]
        remaining_urls = len(urls_to_scrape)
        logging.info(f"Found {remaining_urls} URLs to process")
        
        if urls_to_scrape.empty:
            logging.info("No URLs to process. All URLs are marked as completed.")
            return
        
        scraper = AsyncSEOScraper()
        batch_size = 10  # every batch URL amount
        total_processed = 0
        
        try:
            # dispose URL in batches
            for start_idx in range(0, len(urls_to_scrape), batch_size):
                batch = urls_to_scrape.iloc[start_idx:start_idx + batch_size]
                results = []
                
                for index, row in batch.iterrows():
                    url = row['link']
                    total_processed += 1
                    
                    try:
                        logging.info(f"Processing URL {total_processed}/{remaining_urls} ({(total_processed/remaining_urls)*100:.2f}%): {url}")
                        
                        # crawl
                        result = await scraper.scrape_url(url)
                        results.append(result)
                        logging.info(f"Successfully scraped data for: {url}")
                        
                        # update
                        df.at[index, 'in_progress'] = 1
                        df.to_csv(output_file, index=False)
                        
                        #random wait
                        await asyncio.sleep(random.uniform(10, 15))
                        
                    except Exception as e:
                        logging.error(f"Error processing URL {url}: {str(e)}")
                        results.append({
                            "URL": url,
                            "Status": f"Failed: {str(e)}"
                        })
                
                # save results of every batch
                results_df = pd.DataFrame(results)
                try:
                    if os.path.exists(results_file):
                        existing_results = pd.read_csv(results_file)
                        combined_results = pd.concat([existing_results, results_df], ignore_index=True)
                        combined_results.to_csv(results_file, index=False)
                    else:
                        results_df.to_csv(results_file, index=False)
                    
                    logging.info(f"Saved batch results. Total processed: {total_processed}/{remaining_urls}")
                    
                except Exception as e:
                    logging.error(f"Error saving results file: {str(e)}")
                
                # Wait extra after each batch to avoid being blocked
                await asyncio.sleep(random.uniform(20, 30))
                
                # print status
                print(f"\nProgress: {total_processed}/{remaining_urls} ({(total_processed/remaining_urls)*100:.2f}%)")
                print(f"Latest batch results preview:")
                if not results_df.empty:
                    print(results_df[['URL', 'Title', 'Status']].head())
            
        finally:
            await scraper.close()
        
        logging.info("Processing completed successfully")
        print("\nScraping completed!")
        print(f"Total URLs processed: {total_processed}")
        print(f"Results saved to: {results_file}")
        print(f"Log file: {log_file}")
        
    except Exception as e:
        logging.error(f"An error occurred in main: {str(e)}")
        raise

def main():
    print(f"Starting SEO scraper...")
    print(f"Log file: {log_file}")
    try:
        asyncio.get_event_loop().run_until_complete(run_scraper())
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print(f"Check {log_file} for error details")
        raise

if __name__ == "__main__":
    main()

Starting SEO scraper...
Log file: scraper_20250510_101039.log
2025-05-10 10:10:39,980 - INFO - Reading input file: /Users/baoyunhang/Downloads/top_10_results_filtered.csv
2025-05-10 10:10:39,995 - INFO - Successfully read 5877 rows from input file
2025-05-10 10:10:39,996 - INFO - Added 'in_progress' column to DataFrame
2025-05-10 10:10:39,996 - INFO - Found 5877 URLs to process
2025-05-10 10:10:39,997 - INFO - Processing URL 1/5877 (0.02%): https://www.tslo.com/real-estate-law/commercial-leasing/
2025-05-10 10:10:41,986 - INFO - Successfully scraped data for: https://www.tslo.com/real-estate-law/commercial-leasing/


KeyboardInterrupt: 

2025-05-10 10:10:52,119 - INFO - Processing URL 2/5877 (0.03%): https://www.eastbaybusinesslawyer.com/commercial-lease-review-negotiation-lawyer/
2025-05-10 10:10:52,136 - ERROR - Attempt 1 failed for https://www.eastbaybusinesslawyer.com/commercial-lease-review-negotiation-lawyer/: BrowserContext.new_page: Connection closed while reading from the driver
2025-05-10 10:11:18,518 - ERROR - Error initializing browser: Browser.close: Connection closed while reading from the driver
2025-05-10 10:11:18,520 - ERROR - Error processing URL https://www.eastbaybusinesslawyer.com/commercial-lease-review-negotiation-lawyer/: Browser.close: Connection closed while reading from the driver
2025-05-10 10:11:18,521 - INFO - Processing URL 3/5877 (0.05%): https://www.zfplaw.com/real-estate-transactions/rental-agreements-and-leases/
2025-05-10 10:11:18,528 - ERROR - Attempt 1 failed for https://www.zfplaw.com/real-estate-transactions/rental-agreements-and-leases/: BrowserContext.new_page: Connection close